In [49]:
import pandas as pd
import numpy as np
import requests
import ast 
from datetime import datetime, timedelta

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [50]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, paises):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 

        self.paises = paises

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def cargar_ataques(self, ruta_archivo):

        self.ruta_archivo = ruta_archivo

        df_ataques = pd.read_csv(ruta_archivo, index_col=0)

        lista_paises = []
        
        for k in self.paises.keys():

            lista_paises.append(k.lower())

        df_ataques = df_ataques[df_ataques['country'].isin(lista_paises)]

        return df_ataques


    
    def llamada_API(self, producto):

        self.producto = producto

        df_final = pd.DataFrame()

        for key, value in self.paises.items():
        
            url = f'http://www.7timer.info/bin/api.pl?lon=-{value[1]}&lat={value[0]}&product={producto}&output=json'

            response = requests.get(url=url)
            codigo_estado = response.status_code
            razon_estado = response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            
            df = pd.json_normalize(response.json()["dataseries"])
            df['latitud'] = value[0]
            df['longitud'] = value[1]
            df['pais'] = key
            df_final = pd.concat([df_final,df], axis=0, ignore_index=True) 
         
        return df_final
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_columnas_meteo(self, df_meteo):

        x = df_meteo['wind_profile'].apply(pd.Series)

        for i in range(len(x.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = 'windspeed_' + str(x[i].apply(pd.Series)['layer'][0])

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["speed"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_meteo.insert(i, nombre, valores)

        x = df_meteo['rh_profile'].apply(pd.Series)

        for i in range(len(x.columns)): 

        # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = 'rh_' + str(x[i].apply(pd.Series)['layer'][0] )
            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_meteo.insert(i, nombre, valores)

        return df_meteo

    def meteo_medias_fecha(self, df_meteo):

        df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())

        df_meteo_medias.reset_index(inplace=True)

        df_meteo_medias['pais'] = df_meteo_medias['pais'].apply(lambda x: x.lower())

        hoy = datetime.now()
        
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        
        df_meteo_medias["fecha_descarga"] = hoy

        df_meteo_medias.to_csv('../../datos/meteo_medias.csv')
        df_meteo_medias.to_csv('../../datos/meteo_medias.pkl')

        
        return df_meteo_medias 


    def juntar_dfs(self, df_meteo_medias, df_ataques): 

        df_ataques_meteo =  pd.merge(left = df_ataques, right= df_meteo_medias, how= "left", left_on= "country", right_on= "pais")
        
        #eliminamos columna duplicada
        df_ataques_meteo.drop(columns='pais', axis=1, inplace=True)

        # guardamos los dato
        df_ataques_meteo.to_pickle('../../datos/attacks_meteo.pkl')
        df_ataques_meteo.to_csv('../../datos/attacks_meteo.csv')

        return df_ataques_meteo
    

    def chequear_datos(self, df_ataques_meteo): 
    
        print("Las columnas son:", "\n")
        print(list(df_ataques_meteo.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_ataques_meteo.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_ataques_meteo.isnull().sum() / df_ataques_meteo.shape[0]) *  100)


Comprobamos que nuestra Clase funciona correctamente

In [51]:
paises = {"USA" : [39.7837304,-100.445882], "AUSTRALIA" : [-24.7761086, 134.755], "SOUTH AFRICA" : 
    [-28.8166236, 24.991639], "NEW ZEALAND" : [-41.5000831, 172.8344077], "PAPUA NEW GUINEA" : [-5.6816069, 144.2489081]}
producto = "meteo"
ruta_ataques = '../../datos/attacks_final.csv'

In [52]:
api = Extraccion(paises)

In [53]:
api

In [54]:
df_meteo = api.llamada_API(producto)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [55]:
df_meteo.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 65, 'speed': ...",23,2,10,1016,rain,2,0,90,3,39.78373,-100.445882,USA


In [56]:
df_meteo = api.limpiar_columnas_meteo(df_meteo)

In [57]:
df_meteo

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,13,16,15,15,14,9,7,5,3,0,...,10,1016,rain,2,0,90,3,39.783730,-100.445882,USA
1,12,15,16,15,8,8,7,3,0,-1,...,9,1016,rain,2,0,90,2,39.783730,-100.445882,USA
2,11,15,15,12,8,8,5,-1,-3,-2,...,8,1014,rain,2,0,110,2,39.783730,-100.445882,USA
3,12,15,13,7,7,5,1,-2,-3,-3,...,9,1014,none,2,0,140,2,39.783730,-100.445882,USA
4,12,14,9,6,6,3,1,-1,-2,-2,...,9,1015,none,2,0,165,2,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,11,15,0,1,3,-1,-3,-4,-4,-4,...,9,1013,rain,3,0,90,4,-5.681607,144.248908,PAPUA NEW GUINEA
316,11,13,2,1,3,0,-3,-4,-4,-4,...,11,1012,none,3,0,95,4,-5.681607,144.248908,PAPUA NEW GUINEA
317,10,11,5,1,2,0,-2,-4,-4,-4,...,11,1010,none,3,0,100,4,-5.681607,144.248908,PAPUA NEW GUINEA
318,14,13,11,2,0,-2,-3,-4,-4,-4,...,12,1010,none,3,0,105,4,-5.681607,144.248908,PAPUA NEW GUINEA


In [58]:
df_meteo_medias = api.meteo_medias_fecha(df_meteo)

/tmp/ipykernel_3790/3937975906.py:91: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())


In [59]:
df_meteo_medias

,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,fecha_descarga
0,australia,12.687500,12.156250,9.625000,5.406250,2.656250,1.687500,1.187500,0.687500,0.625000,...,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776109,134.755000,2022-09-18
1,new zealand,11.937500,12.953125,10.015625,6.640625,5.125000,3.203125,4.859375,6.031250,6.296875,...,8.046875,11.734375,1024.046875,2.515625,0.0,3.284456e+167,3.68750,-41.500083,172.834408,2022-09-18
2,papua new guinea,13.734375,11.671875,7.171875,2.796875,0.437500,-0.828125,-3.109375,-3.937500,-4.000000,...,1.625000,11.281250,1010.984375,2.531250,0.0,1.720470e+164,3.68750,-5.681607,144.248908,2022-09-18
3,south africa,12.546875,11.812500,10.921875,9.015625,8.171875,7.406250,6.187500,5.031250,4.390625,...,2.875000,12.406250,1018.109375,5.281250,0.0,5.394555e+157,3.25000,-28.816624,24.991639,2022-09-18
4,usa,10.656250,11.953125,11.000000,9.546875,8.734375,8.640625,7.906250,7.359375,6.093750,...,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882,2022-09-18


In [60]:
df_ataques = api.cargar_ataques(ruta_ataques)

In [61]:
df_ataques.head(1)

,case_number,year,mes,country,type,age,sex,species,fatal
3,1791.00.00,1976,Unknown,australia,Unprovoked,27.28,F,UNKNOWN,y


In [62]:
df_ataques_meteo = api.juntar_dfs(df_meteo_medias, df_ataques)

In [63]:
df_ataques_meteo.sample(10)

,case_number,year,mes,country,type,age,sex,species,fatal,rh_950mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,fecha_descarga
317,2015.09.26,2015,Sep,australia,Unprovoked,7.00,F,UNKNOWN,n,12.687500,...,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776109,134.755000,2022-09-18
1026,2007.11.18,2007,Nov,australia,Provoked,32.00,M,UNKNOWN,n,12.687500,...,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776109,134.755000,2022-09-18
653,2012.06.14.d,2012,Jun,usa,Unprovoked,18.00,F,UNKNOWN,n,10.656250,...,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882,2022-09-18
657,2012.06.12,2012,Jun,australia,Unprovoked,42.00,M,UNKNOWN,n,12.687500,...,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776109,134.755000,2022-09-18
4324,1862.12.22,1862,Dec,new zealand,Unprovoked,28.00,M,UNKNOWN,n,11.937500,...,8.046875,11.734375,1024.046875,2.515625,0.0,3.284456e+167,3.68750,-41.500083,172.834408,2022-09-18
3731,1931.02.10,1931,Feb,australia,Invalid,39.00,M,UNKNOWN,unknown,12.687500,...,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776109,134.755000,2022-09-18
4002,1910.03.00,1910,Mar,usa,Invalid,27.28,M,UNKNOWN,unknown,10.656250,...,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882,2022-09-18
1881,1984.11.08,1984,Nov,south africa,Unprovoked,14.00,M,UNKNOWN,n,12.546875,...,2.875000,12.406250,1018.109375,5.281250,0.0,5.394555e+157,3.25000,-28.816624,24.991639,2022-09-18
3066,1960.04.24,1960,Apr,usa,Unprovoked,48.00,M,WHITE SHARK,n,10.656250,...,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882,2022-09-18
4319,1863.07.09,1863,Jul,usa,Provoked,27.28,M,UNKNOWN,n,10.656250,...,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882,2022-09-18


In [64]:
api.chequear_datos(df_ataques_meteo)

Las columnas son: 

['case_number', 'year', 'mes', 'country', 'type', 'age', 'sex', 'species', 'fatal', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'windspeed_950mb', 'windspeed_900mb', 'windspeed_850mb', 'windspeed_800mb', 'windspeed_750mb', 'windspeed_700mb', 'windspeed_650mb', 'windspeed_600mb', 'windspeed_550mb', 'windspeed_500mb', 'windspeed_450mb', 'windspeed_400mb', 'windspeed_350mb', 'windspeed_300mb', 'windspeed_250mb', 'windspeed_200mb', 'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount', 'snow_depth', 'wind10m.direction', 'wind10m.speed', 'latitud', 'longitud', 'fecha_descarga']
-----------------------------------------
Los tipos de datos que tenemos son: 

case_number           object
year                   int64
mes                   object
country    